로메니아 문제 optimal path, time, cost -> aco, bee, tree, graph

In [18]:
import time
import heapq
import random
from collections import defaultdict

# Romania map data as adjacency list with distances
romania_map = {
    'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],
    'Zerind': [('Oradea', 71), ('Arad', 75)],
    'Oradea': [('Zerind', 71), ('Sibiu', 151)],
    'Sibiu': [('Arad', 140), ('Oradea', 151), ('Fagaras', 99), ('Rimnicu Vilcea', 80)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Rimnicu Vilcea': [('Sibiu', 80), ('Pitesti', 97), ('Craiova', 146)],
    'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
    'Timisoara': [('Arad', 118), ('Lugoj', 111)],
    'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
    'Mehadia': [('Lugoj', 70), ('Drobeta', 75)],
    'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
    'Craiova': [('Drobeta', 120), ('Rimnicu Vilcea', 146), ('Pitesti', 138)],
    'Bucharest': [('Fagaras', 211), ('Pitesti', 101), ('Giurgiu', 90), ('Urziceni', 85)],
    'Giurgiu': [('Bucharest', 90)],
    'Urziceni': [('Bucharest', 85), ('Hirsova', 98), ('Vaslui', 142)],
    'Hirsova': [('Urziceni', 98), ('Eforie', 86)],
    'Eforie': [('Hirsova', 86)],
    'Vaslui': [('Urziceni', 142), ('Iasi', 92)],
    'Iasi': [('Vaslui', 92), ('Neamt', 87)],
    'Neamt': [('Iasi', 87)],
}

# Graph Search algorithm
def graph_search(start, goal):
    start_time = time.time()
    frontier = [(0, start, [])]
    explored = set()

    while frontier:
        cost, state, path = heapq.heappop(frontier)
        if state == goal:
            return path + [state], cost, time.time() - start_time
        if state in explored:
            continue
        explored.add(state)
        for neighbor, step_cost in romania_map[state]:
            if neighbor not in explored:
                heapq.heappush(frontier, (cost + step_cost, neighbor, path + [state]))
    return None, float('inf'), time.time() - start_time

# Tree Search algorithm
def tree_search(start, goal):
    start_time = time.time()
    frontier = [(0, start, [])]

    while frontier:
        cost, state, path = heapq.heappop(frontier)
        if state == goal:
            return path + [state], cost, time.time() - start_time
        for neighbor, step_cost in romania_map[state]:
            heapq.heappush(frontier, (cost + step_cost, neighbor, path + [state]))
    return None, float('inf'), time.time() - start_time

# ACO algorithm
def aco_search(start, goal, iterations=100, alpha=1, beta=5, evaporation=0.3):
    start_time = time.time()
    pheromone = defaultdict(lambda: 1.0)

    def probability(from_node, to_node):
        for neighbor, dist in romania_map[from_node]:
            if neighbor == to_node:
                return (pheromone[(from_node, to_node)] ** alpha) * ((1.0 / dist) ** beta)
        return 0

    best_path = None
    best_cost = float('inf')

    for _ in range(iterations):
        path = [start]
        cost = 0
        visited = set(path)
        current = start

        while current != goal:
            neighbors = [(n, d) for n, d in romania_map[current] if n not in visited]
            if not neighbors:
                break
            probs = [probability(current, n) for n, _ in neighbors]
            total = sum(probs)
            if total == 0:
                break
            probs = [p / total for p in probs]
            chosen = random.choices(neighbors, weights=probs)[0]
            path.append(chosen[0])
            cost += chosen[1]
            visited.add(chosen[0])
            current = chosen[0]

        if current == goal and cost < best_cost:
            best_path = path
            best_cost = cost
        for i in range(len(path) - 1):
            pheromone[(path[i], path[i + 1])] += 1.0
        for edge in pheromone:
            pheromone[edge] *= (1 - evaporation)

    return best_path, best_cost, time.time() - start_time

# Bee algorithm
def bee_search(start, goal, scouts=100, elite_sites=10, iterations=100):
    start_time = time.time()

    def random_path():
        path = [start]
        current = start
        visited = set(path)
        cost = 0
        while current != goal:
            neighbors = [(n, d) for n, d in romania_map[current] if n not in visited]
            if not neighbors:
                break
            next_city, dist = random.choice(neighbors)
            path.append(next_city)
            cost += dist
            visited.add(next_city)
            current = next_city
        return path, cost

    best_path = None
    best_cost = float('inf')
    for _ in range(iterations):
        paths = [random_path() for _ in range(scouts)]
        paths = [p for p in paths if p[0][-1] == goal]
        paths.sort(key=lambda x: x[1])
        if paths and paths[0][1] < best_cost:
            best_path, best_cost = paths[0]

    return best_path, best_cost, time.time() - start_time

# Run all 4 algorithms from Arad to Bucharest
results = {}
results['Graph Search'] = graph_search('Arad', 'Bucharest')
results['Tree Search'] = tree_search('Arad', 'Bucharest')
results['ACO'] = aco_search('Arad', 'Bucharest')
results['Bee'] = bee_search('Arad', 'Bucharest')

import pandas as pd
df = pd.DataFrame([
    {'Algorithm': name, 'Path': ' → '.join(path) if path else 'None', 'Cost': cost, 'Time (s)': f"{duration:.4f}"}
    for name, (path, cost, duration) in results.items()
])
pd.DataFrame(df)

,Algorithm,Path,Cost,Time (s)
0,Graph Search,Arad → Sibiu → Rimnicu Vilcea → Pitesti → Buch...,418,0.0000
1,Tree Search,Arad → Sibiu → Rimnicu Vilcea → Pitesti → Buch...,418,0.0000
2,ACO,Arad → Zerind → Oradea → Sibiu → Rimnicu Vilce...,575,0.0010
3,Bee,Arad → Sibiu → Rimnicu Vilcea → Pitesti → Buch...,418,0.0300
